In [1]:
import dask
import dask.dataframe as dd
import pandas as pd

## Exploring OpenFoodFacts CSV
Source : https://fr.openfoodfacts.org/data/fr.openfoodfacts.org.products.csv

In [2]:
ddf_OFF = dd.read_csv("fr.openfoodfacts.org.products.csv", sep="\t", encoding="utf-8", low_memory=False,
                  #usecols=["code", "url", "quantity", "product_name", "categories_tags", "countries_tags", "ingredients_text", "states_tags", "main_category"],
                  usecols=["code", "product_name", "categories_tags", "main_category"],
                  dtype={'allergens': 'object',
       'cities_tags': 'object',
       'emb_codes': 'object',
       'emb_codes_tags': 'object',
       'first_packaging_code_geo': 'object',
       'generic_name': 'object',
       'ingredients_from_palm_oil_tags': 'object',
       'labels': 'object',
       'labels_fr': 'object',
       'labels_tags': 'object',
       'manufacturing_places': 'object',
       'manufacturing_places_tags': 'object',
       'origins': 'object',
       'origins_tags': 'object',
       'stores': 'object',
       'code': 'object','allergens_fr': 'object',
       'cities': 'object',
       'created_t': 'object',
       'last_modified_t': 'object'})
ddf_OFF.head()

,code,product_name,categories_tags,main_category
0,0000000000017,Vitória crackers,NaN,NaN
1,0000000000031,Cacao,NaN,NaN
2,0000000000123,Sauce Sweety chili 0%,NaN,NaN
3,0000000000291,Mendiants,NaN,NaN
4,0000000000949,Salade de carottes râpées,NaN,NaN


## Exploring Phenix SQL database

Connection via SQL Express en local

In [3]:
import pyodbc
con = pyodbc.connect(Trusted_Connection='yes', driver = '{SQL Server}',server = 'localhost\SQLEXPRESS' , database = 'PHX_D4G')

Envoie de la query SQL --> Dataframe

In [4]:
sql = """select Produits.Ean,Produits.Nom,CategorieProduits.Nom \n
    from Produits \n
    inner join CategorieProduits on Produits.Categorie_CategorieProduit_Id=CategorieProduits.Id \n
    where CategorieProduits.Id<=5;"""
df_Phenix = pd.read_sql(sql,con)


Préparation pour le merge avec la DB OpenFoodFacts 
* Changement du nom de colonne EAN
* Suppression des duplicats dans les produits PHENIX

In [5]:
df_Phenix = df_Phenix.rename(columns = {'Ean':'code'})

print(df_Phenix.shape)
df_Phenix.columns = ['code','Nom','categorie']
df_Phenix = df_Phenix.drop_duplicates()
print(df_Phenix.shape)

(573013, 3)
(322191, 3)


Merge de DB produits unique de Phenix et DB OpenFoodFacts
* Merge des DB
* Renommer les colonnes pour la présentation
* Extraction en CSV

In [6]:
df_OFF_et_Phenix = pd.merge(ddf_OFF.compute(), df_Phenix, on=['code'], how='right')
df_OFF_et_Phenix.columns = ['EAN','OFF_product_name','OFF_categories_tags','OFF_main_category','PHENIX_Nom','PHENIX_categorie']
df_OFF_et_Phenix.to_csv('comparisonDB_OFF_vs_PHENIX.csv')